In [1]:
import dash
import pandas as pd
import plotly.express as px
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate

In [2]:
df = pd.read_csv('cc_data.csv')
#d = ['2021-12-08','2021-12-07','2021-12-06']
#dft = df[df['Date'].isin(d)]
l = [0,1,3,4,5]
dfM = df.iloc[:,l]
dfM = pd.melt(dfM, id_vars=['Date','Date_Time','Diff'],var_name='hash', value_name='rate')
#dfM[dfM['Date']=='2021-12-24']
#df.groupby(by='Date').mean()['Diff'][-1:]
dfM
#df.groupby(by='Date').mean()[-1:]['Calculated_Hash']
#df.groupby(by='Date').mean().filter(items=['2021-12-01'], axis=0)['Calculated_Hash']

,Date,Date_Time,Diff,hash,rate
0,0,0,0.000000,Reported_Hash,0
1,2021-11-21,13:33:51.397040,0.000000,Reported_Hash,30
2,2021-11-21,13:34:53.458088,0.000000,Reported_Hash,30
3,2021-11-21,14:01:40.669087,350.001428,Reported_Hash,30
4,2021-11-21,14:03:51.583825,141.313850,Reported_Hash,30
...,...,...,...,...,...
3169,2022-01-18,00:38:16.200722,1596.558007,Calculated_Hash,44
3170,2022-01-18,00:53:26.559562,435.871093,Calculated_Hash,19
3171,2022-01-18,14:11:01.917901,0.000000,Calculated_Hash,0
3172,2022-01-18,14:26:12.303149,127.905193,Calculated_Hash,15


In [3]:
def inter_func():
  df = pd.read_csv('cc_data.csv')
  l = [0,1,3,4,5]
  df_icb = df.iloc[:,l]
  df_icb = pd.melt(df_icb, id_vars=['Date','Date_Time','Diff'],var_name='hash', value_name='rate')
  return df_icb

df_itv = inter_func()
df_itv

,Date,Date_Time,Diff,hash,rate
0,0,0,0.000000,Reported_Hash,0
1,2021-11-21,13:33:51.397040,0.000000,Reported_Hash,30
2,2021-11-21,13:34:53.458088,0.000000,Reported_Hash,30
3,2021-11-21,14:01:40.669087,350.001428,Reported_Hash,30
4,2021-11-21,14:03:51.583825,141.313850,Reported_Hash,30
...,...,...,...,...,...
3169,2022-01-18,00:38:16.200722,1596.558007,Calculated_Hash,44
3170,2022-01-18,00:53:26.559562,435.871093,Calculated_Hash,19
3171,2022-01-18,14:11:01.917901,0.000000,Calculated_Hash,0
3172,2022-01-18,14:26:12.303149,127.905193,Calculated_Hash,15


In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div([ #main div
    html.Div(children=[ #Graph, Dropdown and Radio Div
    dcc.Graph(id='graph-with-drop'), 
    dcc.Dropdown(
        id='drop-year',
        options=[{'label':i,'value':i} for i in dfM['Date'].unique()],value=dfM['Date'].max()
    ), 
    dcc.Interval(
        id='my_interval',
        disabled=False,
        interval=1*9450000, #15.75 Minutes
        n_intervals=0,
        max_intervals=40),  #40 time 10 hours
    dcc.RadioItems(
        id='radio_coin_hash',
        options=[
            {'label':'Hash Rate','value':'rate'},
            {'label':'Coins','value':'Diff'}
        ],value='rate'#CLOSE RADIO list
    ), #CLOSE RADIO button
        html.Br(),
        ]), #CLOSE GRAPH, DROPDOWN and RADIO Div
    
#----Pie  
    
    html.Div(children=[ #Average hash pie div
        #html.H1('Average Hash Rate'),
        dcc.Graph(id='average_hash_pie')
    ],style={'padding': 10, 'flex': 1, 'display': 'inline-block', 'margin-left':'25px'}), #CLOSE AVERAGE HASH div
    
    html.Div(children=[ #reported hash pie div
        #html.H1('Current Hash Rate'),
        dcc.Graph(id='reported_hash_pie')
    ],style={'padding': 10, 'flex': 1, 'display': 'inline-block', 'margin-left':'75px'}), #CLOSE REPORTED HASH div
    
    html.Div(children=[ #calc hash pie div
        #html.H1('Calculated Hash Rate'),
        dcc.Graph(id='calc_hash_pie')
    ],style={'padding': 10, 'flex': 1, 'display': 'inline-block', 'margin-left':'75px'}), #CLOSE CALC HASH div
    
    html.Br(),

#---Coin    
    html.Div(children=[ #Average coin div
        html.H3('Day_Avg_Coin_Per_Hr'),
        html.Div(id='avg_coin_hour')
    ],style={'padding': 10, 'flex': 1, 'display': 'inline-block', 'margin-left':'75px'}), #CLOSE AVERAGE COIN div
    
    html.Div(children=[ #Days Coin div
        html.H3('Total_Day_Coin'),
        html.Div(title='Todays Coin',id='day_tot_coin')
    ], style={'padding': 10, 'flex': 1, 'display': 'inline-block', 'margin-left':'75px'}), #CLOSE TOTAL COIN div
    
    html.Div(children=[ #Total Coin div
        html.H3('Total_Coin'),
        html.Div(title='Total Coin',id='tot_coin')
    ], style={'padding': 10, 'flex': 1, 'display': 'inline-block', 'margin-left':'75px'}), #CLOSE TOTAL COIN div
    
    html.Div(children=[ #Total Uptime
        html.H3('Day Up Time'),
        html.Div(title='Day Up Time',id='day_uptime')
    ], style={'padding': 10, 'flex': 1, 'display': 'inline-block', 'margin-left':'75px'}) #CLOSE TOTAL UP TIME COIN div
    
]) #close main div

#---Graph call back

@app.callback( #Callback for graph and radio button
    Output('graph-with-drop', 'figure'),
    Input('drop-year', 'value'),
    Input('radio_coin_hash', 'value'),
    Input('my_interval','n_intervals'))
def update_figure(selected_year,cat,num):
    #if num==0:
    #  raise PreventUpdate
    #else:
    df = pd.read_csv('cc_data.csv')
    l = [0,1,3,4,5]
    df_icb = df.iloc[:,l]
    df_icb = pd.melt(df_icb, id_vars=['Date','Date_Time','Diff'],var_name='hash', value_name='rate')
    #df_intv = inter_func()
    filtered_df = df_icb[df_icb.Date == selected_year]
    fig1 = px.bar(filtered_df, x="Date_Time", y=cat,
      color="hash", barmode='group')
    fig1.update_layout(transition_duration=500)

    return (fig1)

#----Pie callback

@app.callback( #Call back for the average rate pie
    Output('average_hash_pie', 'figure'),
    Input('drop-year', 'value'),
    Input('my_interval','n_intervals'))
def update_figure(selected_year, num):
    df = pd.read_csv('cc_data.csv')
    ar=df.groupby(by='Date').mean().filter(items=[selected_year], axis=0)['Calculated_Hash']

    labels = pd.Series([int(ar),'..'])
    values = pd.Series([int(ar),100-int(ar)])

    f = {'Label':labels, 'Value':values}
    dfg = pd.DataFrame(f)

    fig = px.pie(dfg,values='Value',names='Label', height=315, width=315, title='Average_#')
    fig.update_traces(textinfo='none', hole=.5, textfont_size=18)
    fig.update_layout(annotations=[dict(text=str(values[0]),showarrow=False, font_size=20)]) 
    return fig

@app.callback( #Call back for the reported hash rate pie
    Output('reported_hash_pie', 'figure'),
    Input('drop-year', 'value'),
    Input('my_interval','n_intervals'))
def update_figure(selected_year,num):
    df = pd.read_csv('cc_data.csv')
    l = [0,1,3,4,5]
    dfM = df.iloc[:,l]
    dfM = pd.melt(dfM, id_vars=['Date','Date_Time','Diff'],var_name='hash', value_name='rate')
    dfRH = dfM[(dfM['Date']==selected_year) & (dfM['hash']=='Reported_Hash')]
    rh = dfRH[-1:]['rate']

    labels = pd.Series([int(rh),'..'])
    values = pd.Series([int(rh),100-int(rh)])

    f = {'Label':labels, 'Value':values}
    df = pd.DataFrame(f)

    fig = px.pie(df,values='Value',names='Label', height=315, width=315, title='Reported_#')
    fig.update_traces(textinfo='none', hole=.5, textfont_size=18)
    fig.update_layout(annotations=[dict(text=str(values[0]),showarrow=False, font_size=20)]) 
    return fig

@app.callback( #Call back for the reported calc rate pie
    Output('calc_hash_pie', 'figure'),
    Input('drop-year', 'value'),
    Input('my_interval','n_intervals'))
def update_figure(selected_year,num):
    df = pd.read_csv('cc_data.csv')
    l = [0,1,3,4,5]
    dfM = df.iloc[:,l]
    dfM = pd.melt(dfM, id_vars=['Date','Date_Time','Diff'],var_name='hash', value_name='rate')
    dfRH = dfM[(dfM['Date']==selected_year) & (dfM['hash']=='Calculated_Hash')]
    ch = dfRH[-1:]['rate']

    labels = pd.Series([int(ch),'..'])
    values = pd.Series([int(ch),100-int(ch)])

    f = {'Label':labels, 'Value':values}
    df = pd.DataFrame(f)

    fig = px.pie(df,values='Value',names='Label', height=315, width=315, title='Calculated_#')
    fig.update_traces(textinfo='none', hole=.5, textfont_size=18)
    fig.update_layout(annotations=[dict(text=str(values[0]),showarrow=False, font_size=20)]) 
    return fig

#---Coin Calc

@app.callback( #Callback for Total Coins
    Output('avg_coin_hour', component_property='children'),
    Input('drop-year', 'value'),
    Input('my_interval','n_intervals'))
def update_tot_coin(selected_year,num):
    df = pd.read_csv('cc_data.csv')
    ac=df.groupby(by='Date').mean().filter(items=[selected_year], axis=0)['Diff']
    ac=f'{ac[0]:,.3f}'
    return ac

@app.callback( #Callback for Total Coins
    Output('day_tot_coin', component_property='children'),
    Input('drop-year', 'value'),
    Input('my_interval','n_intervals'))
def update_tot_coin(selected_year,num):
    df = pd.read_csv('cc_data.csv')
    dc=df.groupby(by='Date').sum().filter(items=[selected_year], axis=0)['Diff']
    dc=f'{dc[0]:,.3f}'
    return dc

@app.callback( #Callback for Total Coins
    Output('tot_coin', component_property='children'),
    Input('drop-year', 'value'),
    Input('my_interval','n_intervals'))
def update_tot_coin(selected_year,num):
    df = pd.read_csv('cc_data.csv')
    t=df[df['Date']==selected_year][-1:]['Num_Coins']
    t=f'{float(t):,.3f}'
    return t

@app.callback( #Callback for Up Time
    Output('day_uptime', component_property='children'),
    Input('drop-year', 'value'),
    Input('my_interval','n_intervals'))
def update_tot_coin(selected_year,num):
    df = pd.read_csv('cc_data.csv')
    dfC = df[1:].copy()
    dfC['DateTime'] = pd.to_datetime(dfC['Date']+' '+dfC['Date_Time'],format='%Y-%m-%d %H:%M:%S.%f')
    dfC['TimeDiff']=dfC['DateTime'].diff()
    dfC['TimeDiff_Min'] = dfC['TimeDiff'].dt.total_seconds()/60
    ccU = dfC[dfC['TimeDiff_Min']<80].groupby('Date').sum()['TimeDiff_Min'][selected_year]/60
    ccU = f'{ccU:,.3f}'
    return ccU

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Jan/2022 14:49:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 14:49:03] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 14:49:03] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 14:49:03] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [18/Jan/2022 14:49:03] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [18/Jan/2022 14:49:03] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [18/Jan/2022 14:49:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 14:49:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 14:49:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 14:49:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2022 14:49:04] "POST /_dash-update-component HTTP/1.